In [72]:
from astropy.io import fits
import numpy as np
from astropy import units as u


In [73]:
# data read in
low_file = "member.uid___A001_X2fb_X200.ari_l.Betelgeuse_sci.spw0_1_2_3_4_338083MHz.12m.cont.I.pbcor.fits"
high_file = "hr_member.uid___A001_X2de_Xf7.ari_l.Betelgeuse_sci.spw0_1_2_3_4_338086MHz.12m.cont.I.pbcor.fits"

In [74]:
# saving low res file header and data info as variable
hdu_lr = fits.open(low_file)
data_lr = hdu_lr[0].data
header_lr = hdu_lr[0].header
# saving high res file header and data info as variable
hdu_hr = fits.open(high_file)
data_hr = hdu_hr[0].data
header_hr = hdu_hr[0].header

In [75]:
from astropy.wcs import WCS

In [76]:
wcs_hr = WCS(header_hr)
wcs_lr = WCS(header_lr)

Set OBSGEO-B to   -23.022886 from OBSGEO-[XYZ].
Set OBSGEO-H to     5053.796 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


In [77]:
wcs_hr

WCS Keywords

Number of WCS axes: 4
CTYPE : 'RA---SIN'  'DEC--SIN'  'FREQ'  'STOKES'  
CRVAL : 88.79306115045  7.407113577698  338086169196.6  1.0  
CRPIX : 1281.0  1281.0  1.0  1.0  
PC1_1 PC1_2 PC1_3 PC1_4  : 1.0  0.0  0.0  0.0  
PC2_1 PC2_2 PC2_3 PC2_4  : 0.0  1.0  0.0  0.0  
PC3_1 PC3_2 PC3_3 PC3_4  : 0.0  0.0  1.0  0.0  
PC4_1 PC4_2 PC4_3 PC4_4  : 0.0  0.0  0.0  1.0  
CDELT : -1.500000001151e-06  1.500000001151e-06  15815123799.35  1.0  
NAXIS : 2560  2560  1  1

In [78]:
wcs_lr

WCS Keywords

Number of WCS axes: 4
CTYPE : 'RA---SIN'  'DEC--SIN'  'FREQ'  'STOKES'  
CRVAL : 88.7930685989  7.407116442487  338083057315.6  1.0  
CRPIX : 442.0  442.0  1.0  1.0  
PC1_1 PC1_2 PC1_3 PC1_4  : 1.0  0.0  0.0  0.0  
PC2_1 PC2_2 PC2_3 PC2_4  : 0.0  1.0  0.0  0.0  
PC3_1 PC3_2 PC3_3 PC3_4  : 0.0  0.0  1.0  0.0  
PC4_1 PC4_2 PC4_3 PC4_4  : 0.0  0.0  0.0  1.0  
CDELT : -9.999999988573e-06  9.999999988573e-06  15875890977.42  1.0  
NAXIS : 882  882  1  1

In [79]:
from astropy.time import Time
from astropy.coordinates import SkyCoord

In [80]:
# setting proper motion of betelgeuse as variables
pm_ra = 26.42 * u.mas/u.yr
pm_dec = 9.60 * u.mas/u.yr
# setting times epochs were taken as variables
hr_time = Time('2015-11-06')
lr_time = Time('2016-08-16')

In [81]:
# getting center from header of each image
crval_hr = [header_hr['CRVAL1'],header_hr['CRVAL2']]*u.deg
crval_lr = [header_lr['CRVAL1'],header_lr['CRVAL2']]*u.deg

In [82]:
distance = 168*u.parsec

In [93]:
crval_lr[0],crval_lr[1]

(<Quantity 88.7930686 deg>, <Quantity 7.40711644 deg>)

In [83]:
# setting up skycoord variable with low res header values
c = SkyCoord(ra=crval_lr[0],
             dec=crval_lr[1],
             distance=distance,
             pm_ra_cosdec=pm_ra,
             pm_dec=pm_dec,
             obstime=Time(Time('2020-08-22')))

In [84]:
Time('2020-08-22')

<Time object: scale='utc' format='iso' value=2020-08-22 00:00:00.000>

In [85]:
header_hr['DATE']

'2020-11-02T03:57:59.155000'

In [86]:
# applying proper motion correction
c.apply_space_motion(hr_time) 

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    (88.79303312, 7.40710366, 168.)
 (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    (26.41999923, 9.60000211, -1.46260001e-05)>

In [94]:
# updating the low resolution header with the new pm corrected center
header_lr['CRVAL1'] = 88.79303312
header_lr['CRVAL2'] = 7.40710366


In [95]:
c.ra.value

88.7930685989

we've now updated the header.

In [96]:
# checking to make sure low res header is updated
header_lr['CRVAL1'],header_lr['CRVAL2']

(88.79303312, 7.40710366)

In [97]:
#saving new wcs as variable
wcs_lr_updated = WCS(header_lr)

In [102]:
hdu_pm = fits.PrimaryHDU(data=data_lr,header=header_lr)
hdul_pm = fits.HDUList([hdu_pm])
hdul_pm.writeto('lr_pm.fits',overwrite=True)
